In [17]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [18]:
import numpy as np
import pandas as pd

In [19]:
import datetime as dt
from datetime import datetime, timedelta
from pprint import pprint

# Reflect Tables into SQLAlchemy ORM

In [20]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.types import Date

In [21]:
# commented out during tutor session.  looks to be object declarations
# Base = declarative_base()
# class measurement(Base):
#     __tablename__ = "measurement"
#     id = Column(Integer, primary_key=True)
#     station = Column(String)
#     date = Column(Date)
#     prcp = Column(Float)
#     tobs = Column(Float)
    
# class station(Base):
#     __tablename__ = "station"
#     id = Column(Integer, primary_key=True)
#     station = Column(String)
#     name = Column(String)
#     latitude = Column(Float)
#     longitude = Column(Float)
#     elevation = Column(Float)

In [22]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [23]:
# reflect an existing database into a new model
base = automap_base()
# reflect the tables
base.prepare(engine, reflect = True)

# commented out during tutor session.  
# inspector = inspect(engine)
# inspector.get_table_names()

In [24]:
# We can view all of the classes that automap found
base.classes.keys()
# https://www.learnpython.org/en/Classes_and_Objects   (understand difference between object and classes)

['measurement', 'station']

In [25]:
# Save references to each table
measurement = base.classes.measurement
station = base.classes.station
station

sqlalchemy.ext.automap.station

In [26]:
# Create our session (link) from Python to the DB
session = Session(bind=engine)
session

In [27]:
# m_columns = inspector.get_columns("measurement")
# pprint(m_columns)

In [28]:
# s_columns = inspector.get_columns("stations")
# pprint(s_columns)

In [29]:
# station_read = pd.read_sql('station', engine) 
# station_read

In [30]:
# measurement_read = pd.read_sql('measurement', engine) 
# measurement_read

In [31]:
# hawaii_merged_df = pd.merge(station_read, measurement_read, how='inner', on="station")
# hawaii_merged_df.to_csv("all_data.csv")
# hawaii_merged_df


# Exploratory Climate Analysis

In [32]:
cmd = """SELECT date
FROM measurement
"""
print(pd.read_sql(cmd, con=engine))

# SELECT DATEADD(month, -2, '2017/08/25') AS DateAdd;
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# result = session.query(func.strftime("%Y-%m-%d",measurement.date).all()
# # Calculate the date 1 year ago from the last data point in the database

# # Perform a query to retrieve the data and precipitation scores

# # Save the query results as a Pandas DataFrame and set the index to the date column

# # Sort the dataframe by date

# # Use Pandas Plotting with Matplotlib to plot the data

             date
0      2010-01-01
1      2010-01-02
2      2010-01-03
3      2010-01-04
4      2010-01-06
...           ...
19545  2017-08-19
19546  2017-08-20
19547  2017-08-21
19548  2017-08-22
19549  2017-08-23

[19550 rows x 1 columns]


In [33]:
last_date = pd.read_sql("SELECT * FROM measurement WHERE date IN (SELECT max(date) FROM measurement)", con=engine)
print(last_date)

      id      station        date  prcp  tobs
0   2724  USC00519397  2017-08-23  0.00  81.0
1   7635  USC00514830  2017-08-23  0.00  82.0
2  12187  USC00519523  2017-08-23  0.08  82.0
3  19550  USC00516128  2017-08-23  0.45  76.0


In [38]:
another_last_date = session.query(func.max(measurement.date)).first()[0]
another_last_date


'2017-08-23'

In [39]:
type(another_last_date)

str

In [63]:
import datetime
date_time_obj = datetime.datetime.strptime(another_last_date, '%Y-%m-%d')
date_time_obj

datetime.datetime(2017, 8, 23, 0, 0)

In [64]:
type(date_time_obj)
# convert this one to datetime.delta for te

datetime.datetime

In [65]:
type(dt.timedelta(days=365))

datetime.timedelta

In [66]:
# testing purposes
one_year_ago = date_time_obj - timedelta(days=365)
print(one_year_ago)

2016-08-23 00:00:00


In [67]:
one_year_ago = dt.date(2017,8,23) - timedelta(days=365)
print(one_year_ago)

2016-08-23


In [69]:
# Two different ways to get the last year of data

In [70]:
results = session.query(measurement.date, measurement.prcp).filter(measurement.date>=one_year_ago).all()

In [75]:
prcp_data = pd.read_sql("SELECT * FROM measurement WHERE date > '2016-08-23'", con=engine)
print(prcp_data)

         id      station        date  prcp  tobs
0      2365  USC00519397  2016-08-24  0.08  79.0
1      2366  USC00519397  2016-08-25  0.08  80.0
2      2367  USC00519397  2016-08-26  0.00  79.0
3      2368  USC00519397  2016-08-27  0.00  77.0
4      2369  USC00519397  2016-08-28  0.01  78.0
...     ...          ...         ...   ...   ...
2218  19546  USC00516128  2017-08-19  0.09  71.0
2219  19547  USC00516128  2017-08-20   NaN  78.0
2220  19548  USC00516128  2017-08-21  0.56  76.0
2221  19549  USC00516128  2017-08-22  0.50  76.0
2222  19550  USC00516128  2017-08-23  0.45  76.0

[2223 rows x 5 columns]


In [73]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prcp_data[["prcp"]].describe()

,prcp
count,2015.000000
mean,0.176462
std,0.460288
min,0.000000
25%,0.000000
50%,0.020000
75%,0.130000
max,6.700000


In [97]:
# Design a query to show how many stations are available in this dataset?
station_count = session.query(station).count()

station_count

9

In [88]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
activity_count = (session.query(measurement.id, measurement.station, measurement.date, measurement.prcp, measurement.tobs, func.count(measurement.date)).group_by(measurement.station).order_by(func.count(measurement.date).desc()).all())
pprint(activity_count)

[(12188, 'USC00519281', '2010-01-01', 0.15, 70.0, 2772),
 (1, 'USC00519397', '2010-01-01', 0.08, 65.0, 2724),
 (2725, 'USC00513117', '2010-01-01', 0.28, 67.0, 2709),
 (9519, 'USC00519523', '2010-01-01', 0.15, 75.0, 2669),
 (16939, 'USC00516128', '2010-01-01', 0.14, 73.0, 2612),
 (5434, 'USC00514830', '2010-03-01', 0.01, 67.0, 2202),
 (14960, 'USC00511918', '2010-01-01', 0.05, 66.0, 1979),
 (7636, 'USC00517948', '2010-05-03', None, 75.0, 1372),
 (9008, 'USC00518838', '2010-01-01', 0.21, 72.0, 511)]


In [98]:
activity_count_2 = session.query(measurement.id, measurement.station, measurement.date, measurement.prcp, measurement.tobs)
print(activity_count_2)
# activity_count_3 = pd.read_sql(activity_count, con=engine)
# activity_count_3

SELECT measurement.id AS measurement_id, measurement.station AS measurement_station, measurement.date AS measurement_date, measurement.prcp AS measurement_prcp, measurement.tobs AS measurement_tobs 
FROM measurement


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
